# SETUP

In [ ]:
# run IPYNB that imports all the relevant packages and functions
%run ../ms_packages_and_functions.ipynb

In [ ]:
output_dir = pjoin(os.getcwd(), 'processed_data')
data_dir  = pjoin(os.getcwd(), 'raw_data')

In [ ]:
run_path = pjoin(output_dir,'RJ_Decoder_000') # name of folder saved in "processed_data"

if not os.path.exists(run_path):
    os.mkdir(run_path)
    print("created directory: {}".format(run_path))
else:
    print("All files will be saved in: {}".format(run_path))

In [ ]:
fig_dir = pjoin(run_path,'figures')

if not os.path.exists(fig_dir):
    os.mkdir(fig_dir)
    print("created directory: {}".format(fig_dir))
else:
    print("All files will be saved in: {}".format(fig_dir))

In [ ]:
# Paths for Generated FNs
FNs_path_fullTrial = 'processed_data/RJ_FN+Embedding_fullTrial-001/RJ_CenterOut_FN_10ms_fullTrial_flattened.pkl'
FNs_path_temporal = 'processed_data/rj1031209_CenterOut_FN-Flattened_10msBin_with500buffer_MIonly.pkl'
FNs_path_temporalNull = 'processed_data/RJ_nulls-poisson-200ms-003/poisson_FNs_000.pkl'

# Multilayer Perceptron

## Full Trial Decoder

### Get features and targets

In [ ]:
FNs_flattened = loadPickle(FNs_path_fullTrial)

In [ ]:
# trials sorted by time it takes for subject to reach target
trials = loadPickle('processed_data/rj031209_trials_by_cutoff.pkl')

### Run Classifier

In [ ]:

performance_scores = []
runs = 50


features = FNs_flattened['data']
targets  = [x[0] for x in FNs_flattened['FN_info']]

for r in range(runs):

    status("Multilayer Perceptron Run # {}".format(r))

    features_train, features_test, targets_train, targets_test = train_test_split(features,
                                                                                targets)

    clf = MLPClassifier()
    clf.fit(features_train, targets_train)

    score = clf.score(features_test,targets_test)

    performance_scores.append(score)

savePickle(pjoin(run_path,f'performance_scores.pkl'),performance_scores)

In [ ]:
# plt.plot(performance_scores)
# plt.plot([0,len(performance_scores)],[np.mean(performance_scores),np.mean(performance_scores)])

In [ ]:
del FNs_flattened, features, targets, features_train, features_test, targets_train, targets_test, clf

## Temporal FN Decoder

In [ ]:
FNs_flattened = loadPickle(FNs_path_temporal)

In [ ]:
# get min time ?(change later to actual min time, but for now this will do)
min_len = time2bin(1.5,lastBin=True)
min_len

In [ ]:
runs = 5
temporal_performance_score = []

for r in range(runs):

    time_performance_score = []

    for ti in range(min_len):
        status("Temporal Multilayer Perceptron, Time {}, Run # {}".format(ti, r))

        # get features:
        features_idx = [i for i,x in enumerate(np.array(FNs_flattened['FN_info'])) if (x[2]==ti)]
        features = np.array(FNs_flattened['data'])[features_idx]

        # get targets:
        targets = [x[0] for x in np.array(FNs_flattened['FN_info'])[features_idx]]

        features_train, features_test, targets_train, targets_test = train_test_split(features,
                                                                                targets)

        clf = MLPClassifier()
        clf.fit(features_train, targets_train)

        score = clf.score(features_test,targets_test)

        time_performance_score.append(score)


    temporal_performance_score.append(time_performance_score)

save_dir = pjoin(run_path,'FN_performance_score.pkl')
savePickle(save_dir,temporal_performance_score)
        


# Rate Matched FN Decoder

In [ ]:
null_FNs = loadPickle(FNs_path_temporalNull)

FNs_flattened = {'data':[flattenNetwork(W) for x in null_FNs[0] for y in x[0] for W in y[0]],
                'FN_info': [[dir,trial,time] for dir in range(len(null_FNs[0])) 
                                                for trial in range(len(null_FNs[0][dir][0])) 
                                                for time in range(len(null_FNs[0][dir][0][trial][0]))]}

In [ ]:
min_len = time2bin(1.5,lastBin=True)

In [ ]:
runs = 5
null_performance_score = []

for r in range(runs):

    null_time_performance_score = []

    for ti in range(min_len):
        status("Temporal Multilayer Perceptron, Time {}, Run # {}".format(ti, r))

        # get features:
        features_idx = [i for i,x in enumerate(np.array(FNs_flattened['FN_info'])) if (x[2]==ti)]
        features = np.array(FNs_flattened['data'])[features_idx]

        # get targets:
        targets = [x[0] for x in np.array(FNs_flattened['FN_info'])[features_idx]]

        features_train, features_test, targets_train, targets_test = train_test_split(features,
                                                                                targets)

        clf = MLPClassifier()
        clf.fit(features_train, targets_train)

        score = clf.score(features_test,targets_test)

        null_time_performance_score.append(score)


    null_performance_score.append(null_time_performance_score)
        
save_dir = pjoin(run_path,'null_performance_score.pkl')
savePickle(save_dir,null_performance_score)

In [ ]:
sns.set_theme()
plt.figure(figsize=(7,5))
x = [bin2time(x,lastBin=True,buffer=0.5) for x in np.arange(min_len)]


plt.plot(x,np.mean(temporal_performance_score,axis=0),label='FN',linewidth=2)

plt.plot(x,np.mean(null_performance_score,axis=0),label='FN null',linewidth=2)
plt.plot(x,np.ones(len(x))*np.mean(performance_scores),label='static')
plt.plot(x,np.ones(len(x))*(1/8),'k--',label='chance')
plt.legend(frameon=False)
plt.savefig(pjoin(fig_dir,f'FN-FNnull-{runs}runs.png'))

# FR Decoder

In [ ]:
FR_flattened = loadPickle('processed_data/RS_FR_run001/RS_FR_flattened.pkl')

In [ ]:

min_len = time2bin(1.5,lastBin=True)
times = np.arange(0,min_len,1)

runs = 50
FR_performance_score = []

for r in range(runs):

    FR_time_performance_score = []

    for ti in times:
        status("Temporal Multilayer Perceptron, Time {}, Run # {}".format(ti, r))

        # get features:
        features_idx = [i for i,x in enumerate(np.array(FNs_flattened['FN_info'])) if (x[2]==ti)]
        features = np.array(FR_flattened['data'])[features_idx]

        # # get targets:
        targets = [x[0] for x in np.array(FR_flattened['FR_info'])[features_idx]]

        features_train, features_test, targets_train, targets_test = train_test_split(features,
                                                                                targets)

        clf = MLPClassifier()
        clf.fit(features_train, targets_train)

        score = clf.score(features_test,targets_test)

        FR_time_performance_score.append(score)


    FR_performance_score.append(FR_time_performance_score)

save_dir = pjoin(run_path,'FR_performance_score.pkl')
savePickle(save_dir,FR_performance_score)

# FRFN Decoder

In [ ]:

min_len = time2bin(1.5,lastBin=True)
times = np.arange(0,min_len,1)

runs = 50
FRFN_performance_score = []

for r in range(runs):

    FRFN_time_performance_score = []

    for ti in times:
        status("Temporal Multilayer Perceptron, Time {}, Run # {}".format(ti, r))

        # get features:
        features_idx = [i for i,x in enumerate(np.array(FNs_flattened['FN_info'])) if (x[2]==ti)]
        features = [ np.hstack([x,y]) for x,y in zip(np.array(FR_flattened['data'])[features_idx],np.array(FNs_flattened['data'])[features_idx])]

        # # get targets:
        targets = [x[0] for x in np.array(FR_flattened['FR_info'])[features_idx]]

        features_train, features_test, targets_train, targets_test = train_test_split(features,
                                                                                targets)

        clf = MLPClassifier()
        clf.fit(features_train, targets_train)

        score = clf.score(features_test,targets_test)

        FRFN_time_performance_score.append(score)


    FRFN_performance_score.append(FRFN_time_performance_score)
save_dir = pjoin(run_path,'FRFN_performance_score.pkl')
savePickle(save_dir,FRFN_performance_score)